### Cluster Analysis

The goal of this notebook is to explore the data structure for hidden
clusters. Good clusters have high intra-class similarity and low
inter-class similarity. Good clustering produces clusters where points
within a cluster are similar and dissimilar from points in different
clusters. We hope to find good clusters in the data we could exploit by
fitting classification models on individual clusters instead of the
entire dataset. We expect the models for individual clusters to be more
accurate and provide better results than a single model fitted on the
entire dataset.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, OPTICS
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score, silhouette_samples
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import warnings
%matplotlib inline

In [ ]:
# Format output of data frame. 
pd.set_option("precision", 4)
pd.set_option('display.max_columns', None)

In [ ]:
# Load the data with all columns
co2 = pd.read_csv('Data/CAIT Country CO2 Emissions-Energy Sub-Sector.csv', skiprows=[0])
ghg = pd.read_csv('Data/CAIT Country GHG Emissions.csv', skiprows=[0, 1])

In [ ]:
# Drop rows with NA
co2 = co2.dropna()
ghg = ghg.dropna()

In [ ]:
# Create multilevel indexes
co2Index = pd.MultiIndex.from_frame(co2.iloc[:,:2])
ghgIndex = pd.MultiIndex.from_frame(ghg.iloc[:,:2])

In [ ]:
ghg.index = ghgIndex
ghg = ghg.iloc[:,2:]

co2.index = co2Index
co2 = co2.iloc[:,2:-1]

In [ ]:
# Scale data using min-max scaling.
scaler = MinMaxScaler()
scaler.fit(ghg)
ghgScale = scaler.transform(ghg)

In [ ]:
# Scale data using min-max scaling.
scaler = MinMaxScaler()
scaler.fit(co2)
co2Scale = scaler.transform(co2)

## Functions

The following functions will be used throughout the cluster analysis
that follows.

The runClusteringModel function fits the cluster model and returns three
different clustering scores, along with the cluster labels. The three
clustering scores are the Variance Ratio Criterion, Davies-Bouldin
Criterion, and Silhouette Coefficient using the specified distance
measure.

In [ ]:
def runClusteringModel(data, silhouetteScoreMeasure, clusteringModel): 
    """Returns Variance Ratio Criterion, Davies-Bouldin score, Silhouette Coefficient 
    based on given distance measure, and the class labels."""

    # Initialize and fit clustering model
    mdl = clusteringModel
    mdl.fit(data)    

    mdlLabels = mdl.labels_  # Get cluster labels

    VRC = calinski_harabasz_score(data, mdlLabels)
    dBScore = davies_bouldin_score(data, mdlLabels)
    silhouetteScore = silhouette_score(data, mdlLabels, metric=silhouetteScoreMeasure, random_state=210)

    return VRC, dBScore, silhouetteScore, mdlLabels

The runPCAClusteringModel function performs PCA analysis keeping the
number of components for the amount of variance specified by the user as
an input parameter.  
The function then calls runClusteringModel to run
the cluster analysis using the PCA transformed data. It returns the same
information as runClusteringModel plus the number of components needed
to capture the amount of variance specified.

In [ ]:
def runPCAClusteringModel(data, silhouetteScoreMeasure, varCaptured, clusteringModel): 
    """Returns the results of clustering algorithm using PCA transformed data and the number of components
    required to caputure the inputted amount of variance.
    varCaptured is in decimal form to specify the amount of the data's variance the components capture. 
    """

    pca = PCA(n_components=None)

    # Initialize PCA keeping all components and fit to data. 
    pca = PCA(n_components=None)
    pca.fit(data)

    # Find number of components that obtain 95% of variance
    cumVar = pca.explained_variance_ratio_.cumsum().round(decimals=2)
    i = 0
    while True:
        if cumVar[i] >= varCaptured:  # True if at least varCaptured of the variance is captured
            numComp = i + 1  # Component number is index plus 1. 
            break
        i += 1

    # Rerun PCA analysis and transformations keeping only three components. 
    pca = PCA(n_components=numComp)
    pca.fit(data)
    data_pca = pca.transform(data)

    # Call runClusteringModel function with PCA transformed data.
    VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(data_pca, silhouetteScoreMeasure, clusteringModel)
    return VRC, dBScore, silhouetteScore, mdlLabels, numComp

The printClusteringStats function prints the results of the cluster
analysis. The results include the three clustering scores obtain from
runClusteringModel. Additionally, the function prints the number of
clusters produced and some information regarding the top 5 clusters
based on the number of instances associated with them.

In [ ]:
def printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels):
    """Prints clustering results, different measures, number of clusters, and stats of top 5 clusters."""
    print('The Variance Ratio Criterion is equal to ' + str(round(VRC,2)) + '.')
    print('The Davies-Bouldin score is equal to ' + str(round(dBScore,2)) + '.')
    print('The mean Silhouette Coefficient using the ' + silhouetteScoreMeasure + ' distance is equal to ' + str(round(silhouetteScore,2)) + '.')
    print('The model produced ' + str(mdlLabels.max() + 1) + ' clusters.\n')

    #show top five clusters and percentage of data in each
    unique, counts = np.unique(mdlLabels, return_counts=True)
    percent = counts/mdlLabels.size
    clusters = pd.DataFrame({'Clusters': unique, 'Number of Instances': counts, 'Percent of Instances': percent})
    print(clusters.sort_values(by=['Number of Instances'], ascending=False).head(5))
    return

The silhouette_scorer function is the scoring method used for
GridSearchCV function. silhouette_scorer is needed because there is no
built-in option for GridSearchCV to score clusters without the true
class labels, which we do not know in this situation. After all, we're
trying to find unknown clusters in the data.

In [ ]:
def silhouette_scorer(data, clusteringModel, silhouetteScoreMeasure='euclidean'):
    """Returns Silhouette Coefficient to be used in GridSearchCV."""

    # Initialize and fit clustering model
    mdl = clusteringModel
    mdl.fit(data)    

    mdlLabels = mdl.labels_  # Get cluster labels

    numLabels = np.unique(mdlLabels).size 
    numInstances = data.shape[0]

    if numLabels == 1 or numLabels == numInstances:  # True if only one cluster or cluster for each individual instance
        return -1
    else:
        return silhouette_score(data, mdlLabels, metric=silhouetteScoreMeasure, random_state=210)

The graphResults function graphs the results from using different
hyperparameters. It displays the Variance Ratio Criterion,
Davies-Bouldin Index, and Silhouette Coefficient on the Y-axes over the
range of hyperparameter values on the x-axis.

In [ ]:
def graphResults(x, VRCLst, dBScoreLst, silhouetteScoreLst):
    """Produce graph for tuning clustering parameters"""

    # Create graph with number of K on x-axis and measures on y-axes.
    fig,ax = plt.subplots()

    # Formatting Y-axis on left side of plot 
    vrc = ax.plot(x, VRCLst, color='red', marker='o', label='Variance Ratio Criterion')
    ax.set_xlabel("Hyperparameter",fontsize=14)
    ax.set_ylabel("Variance Ratio Criterion",color="red",fontsize=14)

    # Formatting Y-axis on right side of plot
    ax2=ax.twinx()
    dbScore = ax2.plot(x, dBScoreLst,color='blue',marker='o', label='Davies–Bouldin index')
    silScore = ax2.plot(x, silhouetteScoreLst,color='green',marker='o', label='Silhouette Score')
    ax2.set_ylabel("DB & Silhouette Score",color='blue',fontsize=14)

    # Formatting the legend and displaying plot
    lns = vrc+dbScore+silScore
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc='center left', bbox_to_anchor=(1.25, 0.5))
    plt.show()

In [ ]:
def plot_silhouettes(data, clusters, metric='euclidean'):
    """Plots cluster silhouettes."""

    cluster_labels = np.unique(clusters)+1  # Label clusters beginning at 1.
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(data, clusters, metric='cosine')
    c_ax_lower, c_ax_upper = 0, 0
    cticks = []
    for i, k in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[clusters == k]
        c_silhouette_vals.sort()
        c_ax_upper += len(c_silhouette_vals)
        color = cm.jet(float(i) / n_clusters)
        plt.barh(range(c_ax_lower, c_ax_upper), c_silhouette_vals, height=1.0, 
                      edgecolor='none', color=color)

        cticks.append((c_ax_lower + c_ax_upper) / 2)
        c_ax_lower += len(c_silhouette_vals)

    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color="red", linestyle="--") 

    plt.yticks(cticks, cluster_labels)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')

    plt.tight_layout()
    plt.show()

    return

In [ ]:
def tuner(data, model, simMeasure='euclidean', varCaptured = .95):
    # X-axis values and those to be tested for k
    x = np.arange(2, 18)

    # placeholders for evaluation measure scores.
    VRCLst = []
    dBScoreLst = []
    silhouetteScoreLst = []

    # Test different values for K
    for k in x:
        if model == 'runClusteringModel':
            VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(data, simMeasure, KMeans(n_clusters=k, random_state=210))
        else:
            VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(data, simMeasure, varCaptured, KMeans(n_clusters=k, random_state=210))
        VRCLst.append(VRC)
        dBScoreLst.append(dBScore)
        silhouetteScoreLst.append(silhouetteScore)

    graphResults(x, VRCLst, dBScoreLst, silhouetteScoreLst)

## KMeans Clustering

#### GHG

In [ ]:
tuner(ghgScale, 'runClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
tuner(ghgScale, 'runClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
tuner(ghgScale, 'runPCAClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
tuner(ghgScale, 'runPCAClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
cv = [(slice(None), slice(None))]
paramDict = {'n_clusters': np.arange(2, 18)}
grid = GridSearchCV(estimator=KMeans(random_state=210), param_grid=paramDict, 
                  scoring=silhouette_scorer, cv=cv, n_jobs=-1)
grid.fit(ghgScale)
print(grid.best_params_) 

#### CO2

In [ ]:
tuner(co2Scale, 'runClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
tuner(co2Scale, 'runClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
tuner(co2Scale, 'runPCAClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
tuner(co2Scale, 'runPCAClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, KMeans(n_clusters=2, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, KMeans(n_clusters=3, random_state=210))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
cv = [(slice(None), slice(None))]
paramDict = {'n_clusters': np.arange(2, 18)}
grid = GridSearchCV(estimator=KMeans(random_state=210), param_grid=paramDict, 
                  scoring=silhouette_scorer, cv=cv, n_jobs=-1)
grid.fit(co2Scale)
print(grid.best_params_) 

K-Means clustering with and without PCA transformed data did not perform
well. The best Silhouette Coefficient produced was .19. The two clusters
may represent functional and non functional water pumps.

## Density-Based Spatial Clustering of Applications with Noise (DBSCAN)

In [ ]:
def dbTuner(data, model, simMeasure='euclidean', varCaptured = .95):
    # X-axis values and those to be tested for k
    x = np.linspace(.1, .4, 20)

    # placeholders for evaluation measure scores.
    VRCLst = []
    dBScoreLst = []
    silhouetteScoreLst = []

    # Test different values for K
    for eps in x:
        if model == 'runClusteringModel':
            VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(data, simMeasure, DBSCAN(eps=eps))
        else:
            VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(data, simMeasure, varCaptured, DBSCAN(eps=eps))
        VRCLst.append(VRC)
        dBScoreLst.append(dBScore)
        silhouetteScoreLst.append(silhouetteScore)

    graphResults(x, VRCLst, dBScoreLst, silhouetteScoreLst)

#### GHG

In [ ]:
dbTuner(ghgScale, 'runClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.2))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
dbTuner(ghgScale, 'runClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.13))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, DBSCAN(eps=.15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
dbTuner(ghgScale, 'runPCAClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.13))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
dbTuner(ghgScale, 'runPCAClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.13))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, DBSCAN(eps=.15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
cv = [(slice(None), slice(None))]
paramDict = {'eps': np.linspace(.1, .4, 20)}
grid = GridSearchCV(estimator=DBSCAN(), param_grid=paramDict, 
                  scoring=silhouette_scorer, cv=cv, n_jobs=-1)
grid.fit(ghgScale)
print(grid.best_params_) 

#### CO2

In [ ]:
dbTuner(co2Scale, 'runClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.12))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.13))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
dbTuner(co2Scale, 'runClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.12))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, DBSCAN(eps=.13))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
dbTuner(co2Scale, 'runPCAClusteringModel', 'euclidean')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.115))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.125))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
dbTuner(co2Scale, 'runPCAClusteringModel', 'cosine')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.1))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.115))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, DBSCAN(eps=.125))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
cv = [(slice(None), slice(None))]
paramDict = {'eps': np.linspace(.1, .4, 20)}
grid = GridSearchCV(estimator=DBSCAN(), param_grid=paramDict, 
                  scoring=silhouette_scorer, cv=cv, n_jobs=-1)
grid.fit(co2Scale)
print(grid.best_params_) 

GridSearchCV indicates the best eps is 0.1, lower than that indicated by
the three measures graphed above.

Density-Based Spatial Clustering of Applications with Noise is producing
too many small clusters to be useful. Noise is accounting for the
largest share of instances, then there are hundreds of many small
clusters that make up around 1.4% or less of the instances in the data
set.

## Hierarchical Clustering

In [ ]:
def hierTuning(data, model, simMeasure='euclidean', varCaptured = .95, affinity='euclidean', linkage='ward'):

    # X-axis values and those to be tested for k
    x = np.arange(2, 11)

    # placeholders for evaluation measure scores.
    VRCLst = []
    dBScoreLst = []
    silhouetteScoreLst = []

    # Test different values for K
    for numClusters in x:        
        if model == 'runClusteringModel':
            VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(data, simMeasure, AgglomerativeClustering(n_clusters=numClusters, affinity=affinity, linkage=linkage))
        else:
            VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(data, simMeasure, varCaptured, AgglomerativeClustering(n_clusters=numClusters, affinity=affinity, linkage=linkage))

        VRCLst.append(VRC)
        dBScoreLst.append(dBScore)
        silhouetteScoreLst.append(silhouetteScore)

    graphResults(x, VRCLst, dBScoreLst, silhouetteScoreLst)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='cosine', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='euclidean', varCaptured = .95, affinity='euclidean', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='euclidean', varCaptured = .95, affinity='euclidean', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='euclidean', varCaptured = .95, affinity='euclidean', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='cosine', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='cosine', varCaptured = .95, affinity='cosine', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='cosine', varCaptured = .95, affinity='cosine', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
hierTuning(ghgScale, 'runPCAClusteringModel', simMeasure='cosine', varCaptured = .95, affinity='cosine', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

#### CO2

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='cosine', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=5, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=5, affinity='cosine', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runClusteringModel', simMeasure='cosine', affinity='cosine', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, AgglomerativeClustering(n_clusters=9, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='euclidean', affinity='euclidean', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='cosine', affinity='euclidean', linkage='ward')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='cosine', affinity='cosine', linkage='complete')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='complete'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='cosine', affinity='cosine', linkage='average')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='average'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
hierTuning(co2Scale, 'runPCAClusteringModel', simMeasure='cosine', affinity='cosine', linkage='single')

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='single'))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

## OPTICS

#### GHG

In [ ]:
def opticsTuning(data, model, simMeasure='euclidean', varCaptured = .95):
    # X-axis values and those to be tested for k
    x = np.arange(5, 51, 5)

    # placeholders for evaluation measure scores.
    VRCLst = []
    dBScoreLst = []
    silhouetteScoreLst = []

    # Test different values for K
    for numSamples in x:
        if model == 'runClusteringModel':
            VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(data, simMeasure, OPTICS(min_samples=numSamples))
        else:
            VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(data, simMeasure, varCaptured, OPTICS(min_samples=numSamples))
        VRCLst.append(VRC)
        dBScoreLst.append(dBScore)
        silhouetteScoreLst.append(silhouetteScore)

    graphResults(x, VRCLst, dBScoreLst, silhouetteScoreLst)

#### GHG

In [ ]:
opticsTuning(ghgScale, 'runClusteringModel', simMeasure='euclidean')

The Davies-Bouldin Index and Silhouette Score agree the default minimum
samples of 5 is the best value for the hyperparameter. The Variance
Ratio Criterion continues to rise throughout the entire range tested but
appears to be plateauing. Due to the high computational cost of running
the OPTICS method, only the default value for min_samples was tested,
which is the value two of the three measures agreed to be the best
hyperparameter.

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, OPTICS(min_samples=10))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, OPTICS(min_samples=40))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
opticsTuning(ghgScale, 'runClusteringModel', simMeasure='cosine')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, OPTICS(min_samples=10))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(ghgScale, silhouetteScoreMeasure, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
opticsTuning(ghgScale, 'runPCAClusteringModel', simMeasure='euclidean')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, OPTICS(min_samples=5))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, OPTICS(min_samples=10))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
opticsTuning(ghgScale, 'runPCAClusteringModel', simMeasure='cosine')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, OPTICS(min_samples=10))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(ghgScale, silhouetteScoreMeasure, .95, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(ghgScale, mdlLabels)

#### CO2

In [ ]:
opticsTuning(co2Scale, 'runClusteringModel', simMeasure='euclidean')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=5))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'euclidean'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=30))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
opticsTuning(co2Scale, 'runClusteringModel', simMeasure='cosine')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=5))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=20))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels = runClusteringModel(co2Scale, silhouetteScoreMeasure, OPTICS(min_samples=30))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
opticsTuning(co2Scale, 'runPCAClusteringModel', simMeasure='euclidean')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=5))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=30))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
opticsTuning(co2Scale, 'runPCAClusteringModel', simMeasure='cosine')

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=5))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=15))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

In [ ]:
warnings.filterwarnings('ignore')
silhouetteScoreMeasure = 'cosine'
VRC, dBScore, silhouetteScore, mdlLabels, numComp = runPCAClusteringModel(co2Scale, silhouetteScoreMeasure, .95, OPTICS(min_samples=30))
printClusteringStats(VRC, dBScore, silhouetteScore, silhouetteScoreMeasure, mdlLabels)

In [ ]:
plot_silhouettes(co2Scale, mdlLabels)

The OPTICS measure did not produce useful clusters. Almost a third of
the instances are classified as noise and none of the almost 3,300
clusters contain more than a percent of the instances.

None of the clustering methods achieved a small number of useful
clusters with good clustering scoring. The methods either produced too
many small clusters with a significant amount of instances classified as
noise or a small number of clusters with poor scoring results.
Therefore, fitting classification models on the resulting clusters would
not be useful.